In [85]:
import pandas as pd

In [41]:
# Load the Excel file
excel_file = pd.ExcelFile('data/Basketball_dataset.xlsx')

# Dictionary to hold data from each sheet
sheets_dict = {}

# Read each sheet into a DataFrame
for sheet_name in excel_file.sheet_names[1:]:
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    df = df.dropna()
    df['Opponent'] = df['Opponent'].apply(lambda s: s.split('(')[0].strip())
    df['Team'] = sheet_name.split('(')[0].strip()

    sheets_dict[sheet_name] = df

print(sheets_dict)


# # To find unique keys
# unique_teams = set()
# for df in sheets_dict.values():
#     data = df.dropna()
#     data = data.apply(lambda s: s.split('(')[0].strip())
#     unique_teams.update(data)

# print(len(unique_teams))

# for t in sorted(list(unique_teams)):
#     print(t)


{'UConn(1)':      G       Date    Type                  Opponent W/L   Tm  Opp   Team
0    1 2023-11-06     REG          Northern Arizona   W   95   52  UConn
1    2 2023-11-11     REG                 Stonehill   W  107   67  UConn
2    3 2023-11-14     REG  Mississippi Valley State   W   87   53  UConn
3    4 2023-11-19     REG                   Indiana   W   77   57  UConn
4    5 2023-11-20     REG                     Texas   W   81   71  UConn
5    6 2023-11-24     REG                 Manhattan   W   90   60  UConn
6    7 2023-11-27     REG             New Hampshire   W   84   64  UConn
7    8 2023-12-01     REG                    Kansas   L   65   69  UConn
8    9 2023-12-05     REG            North Carolina   W   87   76  UConn
9   10 2023-12-09     REG       Arkansas-Pine Bluff   W  101   63  UConn
10  11 2023-12-15     REG                   Gonzaga   W   76   63  UConn
11  12 2023-12-20     REG                Seton Hall   L   60   75  UConn
12  13 2023-12-23     REG             

In [137]:
all_data = sheets_dict.copy()

all_dfs = list(all_data.values())
full_df = pd.concat(all_dfs, ignore_index=True)
full_df = full_df.drop(['G', 'W/L', 'Unnamed: 4'], axis=1)
full_df = full_df.dropna()

full_df = full_df.replace('Fla Atlantic', 'Florida Atlantic')
full_df = full_df.replace('Connecticut', 'UConn')
full_df = full_df.replace('Washington St', 'Washington State')
full_df = full_df.replace('Utah St', 'Utah State')
full_df = full_df.replace('South Dakota St', 'South Dakota State')
full_df = full_df.replace("Saint Peter's", "Saint Peters")
full_df = full_df.replace("Saint Mary's", "Saint Marys")
full_df = full_df.replace("Oregan", "Oregon")
full_df = full_df.replace("NCState", "NC State")
full_df = full_df.replace('Mississippi St.', 'Mississippi State')
full_df = full_df.replace('Michigan St.', 'Michigan State')
full_df = full_df.replace('McNeese', 'McNeese State')
full_df = full_df.replace('Long Beach St', 'Long Beach State')
full_df = full_df.replace('IowaSt', 'Iowa State')
full_df = full_df.replace('Colorado St', 'Colorado State')
full_df = full_df.replace('Col. of Charleston', 'College of Charleston')
full_df = full_df.replace('Boise St', 'Boise State')
full_df = full_df.replace('IowaSt', 'Iowa State')
full_df = full_df.replace('IowaSt', 'Iowa State')
full_df = full_df.replace('IowaSt', 'Iowa State')




def reorder_teams(row): 
    if row['Team'] > row['Opponent']:
        row['Team'], row['Opponent'] = row['Opponent'], row['Team'] 
        row['Tm'], row['Opp'] = row['Opp'], row['Tm'] 
    return row 
    
# Apply the function to each row 

# put alphabet first
full_df = full_df.apply(reorder_teams, axis=1)
full_df['Diff'] = full_df['Tm'] - full_df['Opp']  # calc diff

full_df = full_df.dropna()
full_df = full_df.drop_duplicates()

grouped = full_df.groupby(['Team', 'Opponent']) 


# Mean the numerical columns 
full_df = grouped.agg({col: 'mean' for col in ['Diff', 'Tm', "Opp"]}).reset_index()

full_df

# print(full_df)


# full_df['Date'] = pd.to_datetime(full_df['Date'])  # conv. to dt



all_teams = pd.concat([full_df['Team'], full_df['Opponent']])
print(len(all_teams.unique()))

full_df.sort_values(by='Team')
# 

423


,Team,Opponent,Diff,Tm,Opp
0,Abilene Christian,Grand Canyon,-2.0,71.5,73.5
1,Abilene Christian,NC State,-20.0,64.0,84.0
2,Abilene Christian,Western Ky,-2.0,84.0,86.0
3,Adams State,Colorado State,-45.0,61.0,106.0
4,Air Force,Boise State,-34.5,52.0,86.5
...,...,...,...,...,...
1480,Washington,Washington State,1.5,80.5,79.0
1481,Weber State,Yale,10.0,75.0,65.0
1482,Western Illinois,Wisconsin,-22.0,49.0,71.0
1483,Western Ky,Wichita State,-10.0,61.0,71.0


In [138]:

# Combine 'Team' and 'Opponent' columns to count games played
all_teams = pd.concat([full_df['Team'], full_df['Opponent']])

# Count the number of games each team has played
team_game_counts = all_teams.value_counts()

print(team_game_counts)

# Filter teams that have played more than one game
teams_played_more_than_one_game = team_game_counts[team_game_counts > 1].index
print(teams_played_more_than_one_game)

# Filter the DataFrame to keep only rows where 'Team' and 'Opponent' have played more than one game
full_df_filtered = full_df[full_df['Team'].isin(teams_played_more_than_one_game) & full_df['Opponent'].isin(teams_played_more_than_one_game)]

full_df_filtered
# full_df


NC State                 31
Alabama                  31
Iowa State               30
Tennessee                30
Texas                    29
                         ..
Albany                    1
Adams State               1
St. Katherine             1
Trinity Baptist           1
Vermont State–Johnson     1
Name: count, Length: 423, dtype: int64
Index(['NC State', 'Alabama', 'Iowa State', 'Tennessee', 'Texas', 'Houston',
       'Duquesne', 'Duke', 'Clemson', 'Kansas',
       ...
       'Massachusetts', 'UNC Wilmington', 'Hofstra', 'South Carolina Upstate',
       'UC San Diego', 'UC Davis', 'Louisiana-Monroe', 'Appalachian State',
       'Seattle', 'Southeast Missouri State'],
      dtype='object', length=329)


,Team,Opponent,Diff,Tm,Opp
0,Abilene Christian,Grand Canyon,-2.0,71.5,73.5
1,Abilene Christian,NC State,-20.0,64.0,84.0
2,Abilene Christian,Western Ky,-2.0,84.0,86.0
4,Air Force,Boise State,-34.5,52.0,86.5
5,Air Force,Colorado State,-9.0,71.0,80.0
...,...,...,...,...,...
1480,Washington,Washington State,1.5,80.5,79.0
1481,Weber State,Yale,10.0,75.0,65.0
1482,Western Illinois,Wisconsin,-22.0,49.0,71.0
1483,Western Ky,Wichita State,-10.0,61.0,71.0


In [139]:
import pickle

with open('./data/no_singletons_data.pkl', 'wb') as f: 
    pickle.dump(full_df_filtered, f)